Use BigQuery to load and prepare data for machine learning:

Setup

In [1]:
#inputs:

project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'flash-datum-368412'

In [2]:
REGION = 'us-west1'
EXPERIMENT = '01'
SERIES = '01'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud'

# Data source for this series of notebooks
BQ_SOURCE = 'bigquery-public-data.ml_datasets.ulb_fraud_detection'

In [3]:
#packages
from google.cloud import bigquery
from google.cloud import storage

In [4]:
#clients
bq = bigquery.Client(project = PROJECT_ID)
gcs = storage.Client(project = PROJECT_ID)

In [5]:
#parameters:

BUCKET = PROJECT_ID


In [6]:
#Store the Source Data in GCS Storage Bucket
file = f"{SERIES}/{EXPERIMENT}/data/{BQ_TABLE}.csv"


In [7]:
bucketDef = gcs.bucket(BUCKET)
if storage.Blob(bucket = bucketDef, name = file).exists(gcs):
    print(f'The file has already been created at: gs://{bucketDef.name}/{file}')
else:
    source = bigquery.TableReference.from_string(BQ_SOURCE)
    extract = bq.extract_table(source = source, destination_uris = [f'gs://{bucketDef.name}/{file}'])
    print('Creating the export ...')
    extract.result()
    print(f'Exported the table to: gs://{bucketDef.name}/{file}')

Creating the export ...
Exported the table to: gs://flash-datum-368412/01/01/data/fraud.csv


In [8]:
print(f'Review the files in the console here:\nhttps://console.cloud.google.com/storage/browser/{PROJECT_ID}/{SERIES};tab=objects&project={PROJECT_ID}')


Review the files in the console here:
https://console.cloud.google.com/storage/browser/flash-datum-368412/01;tab=objects&project=flash-datum-368412


In [9]:
#Create BigQuery Dataset
#List BigQuery datasets in the project:

datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

In [10]:
ds = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")
ds.location = REGION
ds.labels = {'experiment': f'{EXPERIMENT}'}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

In [11]:
datasets = list(bq.list_datasets())
for d in datasets:
    print(d.dataset_id)

fraud


In [12]:
#Create BigQuery Table
from google.cloud.exceptions import NotFound
try:
    table = bq.get_table(f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
    if table:
        print(f'The table already exists: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')
except NotFound as error:
    print(f'Creating Table ...')
    destination = bigquery.TableReference.from_string(f"{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}")
    job_config = bigquery.LoadJobConfig(
        write_disposition = 'WRITE_TRUNCATE',
        source_format = bigquery.SourceFormat.CSV,
        autodetect = True,
        labels = {'experiment': f'{EXPERIMENT}'}
    )
    job = bq.load_table_from_uri(f"gs://{bucketDef.name}/{file}", destination, job_config = job_config)
    job.result()
    print(f'Finished creating table: {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}')

Creating Table ...
Finished creating table: flash-datum-368412.fraud.fraud


In [13]:
#Retrieve and Review a Sample From The Table:
query = f"""
SELECT *
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}` TABLESAMPLE SYSTEM (1 PERCENT)
#LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,820,-0.937481,0.401649,1.882689,-0.362001,0.751088,-0.899262,0.880557,-0.181650,-0.211657,...,-0.001757,0.097379,-0.324050,0.436521,0.509674,0.454116,-0.201804,-0.175439,0.00,0
1,3252,-0.734152,1.097267,2.595262,1.434062,-0.058120,0.115607,0.350951,0.129708,-0.472432,...,0.058004,0.238210,-0.104050,0.401831,-0.330435,-0.314001,0.012519,0.158218,0.00,0
2,4696,-1.287907,1.867758,2.893783,2.746577,-0.479992,0.941416,-0.400340,0.356862,0.835515,...,0.041613,0.786050,-0.219993,0.052397,0.098050,0.414711,0.691664,0.338954,0.00,0
3,7319,1.316476,-0.762287,1.147227,-0.537621,-1.180770,0.514797,-1.482598,0.269653,1.035685,...,0.202063,0.901793,-0.099016,-0.320577,0.359021,-0.049807,0.033547,-0.000639,0.00,0
4,7436,1.043965,0.323669,1.676721,2.740750,-0.413087,1.011917,-0.783481,0.330402,1.025439,...,-0.149857,0.003494,0.066054,-0.022276,0.196057,-0.072153,0.033481,0.017221,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,154599,0.667714,3.041502,-5.845112,5.967587,0.213863,-1.462923,-2.688761,0.677764,-3.447596,...,0.329760,-0.941383,-0.006075,-0.958925,0.239298,-0.067356,0.821048,0.426175,6.74,1
284803,90676,-2.405580,3.738235,-2.317843,1.367442,0.394001,1.919938,-3.106942,-10.764403,3.353525,...,10.005998,-2.454964,1.684957,0.118263,-1.531380,-0.695308,-0.152502,-0.138866,6.99,1
284804,34634,0.333499,1.699873,-2.596561,3.643945,-0.585068,-0.654659,-2.275789,0.675229,-2.042416,...,0.469212,-0.144363,-0.317981,-0.769644,0.807855,0.228164,0.551002,0.305473,18.96,1
284805,96135,-1.952933,3.541385,-1.310561,5.955664,-1.003993,0.983049,-4.587235,-4.892184,-2.516752,...,-1.998091,1.133706,-0.041461,-0.215379,-0.865599,0.212545,0.532897,0.357892,18.96,1


In [14]:
#Check out this table in BigQuery Console:
print(f"Direct Link To This Project In BigQuery:\nhttps://console.cloud.google.com/bigquery?project={PROJECT_ID}")


Direct Link To This Project In BigQuery:
https://console.cloud.google.com/bigquery?project=flash-datum-368412


In [15]:
#Review Data in BigQuery
#Additional SQL queries could be used to review the data. This section shows moving the table to a Pandas dataframe for local review in Python:

query = f"""
SELECT Class
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
"""
df = bq.query(query = query).to_dataframe()

In [16]:
df['Class'].value_counts()


0    284315
1       492
Name: Class, dtype: Int64

In [17]:
df['Class'].value_counts(normalize=True)


0    0.998273
1    0.001727
Name: Class, dtype: Float64

In [18]:
#Prepare Data for Analysis
query = f"""
CREATE TABLE IF NOT EXISTS `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`)
SELECT *,
    CASE 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN" 
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [19]:
(job.ended-job.started).total_seconds()


5.707

In [20]:
#Review the test/train split:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
GROUP BY splits
"""
bq.query(query = query).to_dataframe()


,splits,Count,Percentage
0,VALIDATE,28464,9.994136
1,TRAIN,227731,79.959762
2,TEST,28612,10.046101


In [21]:
#Retrieve a subset of the data to a Pandas dataframe:

query = f"""
SELECT * 
FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [22]:
data.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,128681,1.910299,0.683046,-1.032632,3.743651,0.745363,-0.205005,0.533400,-0.141715,-1.437747,...,0.103745,0.002786,0.173663,0.000660,-0.060180,-0.070606,0.0,0,0cd68899-5d47-4a14-8eca-fec033554389,TEST
1,122388,1.925515,0.301887,-0.561146,3.678203,0.615088,1.134870,-0.252903,0.191704,-0.534900,...,-0.072918,-1.072191,0.241449,0.179955,-0.003328,-0.066880,0.0,0,071cb1d5-54b4-443b-919e-3ea2514567fa,TEST
2,43562,-1.170485,0.080223,3.244079,3.557925,0.135464,-0.047191,-0.700745,0.303564,-0.560093,...,-0.171872,1.011799,0.308444,0.461594,0.109793,0.116612,0.0,0,fca8eb8c-0e24-4b37-8cf4-1fa1b5f35a7c,TEST
3,79041,0.152846,0.344174,1.891107,1.214420,-1.001368,0.102999,-0.473924,0.339047,0.349022,...,0.205942,0.541302,-1.669915,0.304682,0.292735,0.251891,0.0,0,6d938266-bcc1-4661-80ed-a3f32d9a9f73,TEST
4,53826,-0.709775,0.716362,0.095709,-0.651650,0.689730,-0.293337,0.345172,0.495644,-0.442210,...,-0.233728,-0.810944,-0.318011,0.328328,-0.020087,0.046195,0.0,0,7e63fef8-6e29-4e53-90f1-0f119d1bab26,TEST
